# Installation and environment setup

In [ ]:
!pip install -qqq r2r # may throw some dependency errors, but it should be okay

In [ ]:
import os
import asyncio
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
asyncio.get_event_loop().set_debug(True)

%autoawait

# Setting environment variables
os.environ['OPENAI_API_KEY'] = 'sk-...'

# Setting postgres db env variables. You can also set it via supabase.com
os.environ['POSTGRES_USER'] = ''
os.environ['POSTGRES_PASSWORD'] = ''
os.environ['POSTGRES_HOST'] = ''
os.environ['POSTGRES_PORT'] = ''
os.environ['POSTGRES_DBNAME'] = ''
os.environ['POSTGRES_VECS_COLLECTION'] = ''

# Quick Introduction

R2R is a powerful library that offers both methods and a REST API for document ingestion, Retrieval-Augmented Generation (RAG), evaluation, and additional features like observability, evaluation, analytics, and document management. It supports configurable vector search and RAG out of the box.



## Downloading R2R dataset sample

In [ ]:
filenames = [
  # .txt
    "aristotle.txt",
    "aristotle_v2.txt",
    "got.txt",
  # .pdf
    "lyft_2021.pdf", # 250 page document
    "uber_2021.pdf", # 300 page document
  # .html
    "pg_essay_1.html",
    "pg_essay_2.html",
    "pg_essay_3.html",
    "pg_essay_4.html",
    "pg_essay_5.html",
  # .mp3
    "sample.mp3",
    "sample2.mp3",
  # .png
    "screen_shot.png"
  # and more...
]


for filename in filenames:
  if not os.path.exists(os.path.join("data", filename)):
    !wget -q https://raw.githubusercontent.com/SciPhi-AI/R2R/main/r2r/examples/data/$filename -P data/

file_paths = [
    os.path.join("data", filename)
    for filename in filenames
]

## Creating an R2R app object

In [ ]:
from r2r import R2RAppBuilder

r2r_app = R2RAppBuilder().build()

2024-06-24 22:15:46,478 - INFO - r2r.core.providers.vector_db_provider - Initializing VectorDBProvider with config extra_fields={} provider='pgvector' collection_name='demo_vecs'.
2024-06-24 22:15:47,600 - INFO - r2r.core.providers.embedding_provider - Initializing EmbeddingProvider with config extra_fields={'text_splitter': {'type': 'recursive_character', 'chunk_size': 512, 'chunk_overlap': 20}} provider='openai' base_model='text-embedding-3-small' base_dimension=512 rerank_model=None rerank_dimension=None rerank_transformer_type=None batch_size=128.
2024-06-24 22:15:50,601 - INFO - r2r.core.providers.llm_provider - Initializing LLM provider with config: extra_fields={} provider='litellm'


R2RApp.__init__, config =  <r2r.main.assembly.config.R2RConfig object at 0x7b261639ef20>


## Ingesting documents into the DB

In [ ]:
# creating R2R documents and ingesting them into the DB
from r2r import Document, generate_id_from_label

documents = []
for file_path in file_paths:

  file_type = file_path.split(".")[-1]
  file_name = file_path.split("/")[-1]
  file_data = open(file_path, "rb").read()

  documents.append(
      Document(
          id = generate_id_from_label(file_name), # creates a unique ID
          type = file_type,
          data = file_data,
          metadata = {
              "source": file_path,
              "title": file_name,
          }
      )
  )

  print(f"Created document for: {file_name}")

# ingest documents into db
# if already ingested, use the update_documents endpoint
ret = r2r_app.ingest_documents(documents)
# ret = r2r_app.update_documents(documents)

print(ret)

Created document for: aristotle.txt
Created document for: aristotle_v2.txt
Created document for: got.txt
Created document for: lyft_2021.pdf
Created document for: uber_2021.pdf
Created document for: pg_essay_1.html
Created document for: pg_essay_2.html
Created document for: pg_essay_3.html
Created document for: pg_essay_4.html
Created document for: pg_essay_5.html
Created document for: sample.mp3
Created document for: sample2.mp3
Created document for: screen_shot.png
{'processed_documents': ["Document 'aristotle.txt' processed successfully.", "Document 'aristotle_v2.txt' processed successfully.", "Document 'got.txt' processed successfully.", "Document 'lyft_2021.pdf' processed successfully.", "Document 'uber_2021.pdf' processed successfully.", "Document 'pg_essay_1.html' processed successfully.", "Document 'pg_essay_2.html' processed successfully.", "Document 'pg_essay_3.html' processed successfully.", "Document 'pg_essay_4.html' processed successfully.", "Document 'pg_essay_5.html' pr

## R2R vector similarity search

In [ ]:
from r2r import VectorSearchSettings

# search_query = "Who was Aristotle?"
# search_query = "Who was the Kingslayer?"
search_query = "What was uber's profit in 2020?"

search_settings = VectorSearchSettings(
                    search_filters=None,
                    search_limit=10,
                    do_hybrid_search=False,
                )

retrieved_chunks = r2r_app.search(search_query, search_settings)
for chunk in retrieved_chunks['vector_search_results']:
    score = round(chunk['score'], 2)
    text = chunk['metadata']['text']
    print(f"Score: {score}\nText: {text}\n{'-'*40}")

/usr/local/lib/python3.10/dist-packages/r2r/vecs/collection.py:641: UserWarning: Query does not have a covering index for cosine_distance. See Collection.create_index
  warnings.warn(


Score: 0.74
Text: Net
 loss attributable to Uber Technologies, Inc. was $496 million, a 93% improvement year-over-year, driven by a $1.6 billion pre-tax gain on the sale of ourATG
 Business to Aurora, a $1.6 billion pre-tax  net benefit relating to Ubers equity investments, as  well as reductions in our fixed cost structure and increasedvariable cost effi
ciencies. Net loss attributable to Uber Technologies, Inc. also included $1.2 billion of stock-based compensation expense.Adjusted
----------------------------------------
Score: 0.67
Text: the year ended December 31, 2020, gain on business divestitures, net represented a $154 million gain on the sale of our Uber Eats India operationsto Zomato
 recognized in the first quarter of 2020 and a $77 million gain on the sale of our European Freight Business to sennder GmbH (Sennder) recognized inthe fourth quarter of 2020, pa
rtially offset by a $27 million loss on the sale of our JUMP operations to Lime recognized in the second quarter of 2

## R2R Retrieval Augmented Generation

In [ ]:
from r2r import GenerationConfig

rag_query = "Who was Aristotle?"
# rag_query = "What was uber's profit in 2020?"

rag_generation_config = GenerationConfig(model="gpt-4o", stream=False)

rag_result = r2r_app.rag(
            query=rag_query,
            rag_generation_config=rag_generation_config,
        )

rag_result[0].choices[0].message.content

'Aristotle (Greek: Ἀριστοτέλης Aristotélēs, pronounced [aristotélɛːs]; 384–322 BC) was an Ancient Greek philosopher and polymath. His writings cover a broad range of subjects spanning the natural sciences, philosophy, linguistics, economics, politics, psychology, and the arts. As the founder of the Peripatetic school of philosophy in the Lyceum in Athens, he began the wider Aristotelian tradition that followed, which set the groundwork for the development of modern science [1], [2], [3], [4], [5], [6], [7], [8], [9], [10].'

========================================================

========================================================

========================================================

========================================================

# Document Management

## Documents Overview

R2R allows retrieval of high-level document information stored in a relational table within the PostgreSQL database. To fetch this information:

In [ ]:
# view

all_docs = r2r_app.documents_overview()

# Returns a `DocumentInfo` object -
# class DocumentInfo(BaseModel):
#     """Base class for document information handling."""

#     document_id: uuid.UUID
#     version: str
#     size_in_bytes: int
#     metadata: dict

#     user_id: Optional[uuid.UUID] = None
#     title: Optional[str] = None
#     created_at: Optional[datetime] = None
#     updated_at: Optional[datetime] = None

for doc in all_docs:
  print(doc.metadata['title'])
  print(doc)

aristotle_v2.txt
document_id=UUID('dbf69e0a-3d41-5140-b20d-017cd6c01ca7') version='v0' size_in_bytes=7443 metadata={'title': 'aristotle_v2.txt', 'source': 'data/aristotle_v2.txt', 'user_id': None} user_id=None created_at=datetime.datetime(2024, 6, 24, 20, 54, 30, 606372, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2024, 6, 24, 22, 21, 24, 387485, tzinfo=datetime.timezone.utc)
got.txt
document_id=UUID('ba77307d-6c8a-549f-812a-3558697e2842') version='v0' size_in_bytes=9491 metadata={'title': 'got.txt', 'source': 'data/got.txt', 'user_id': None} user_id=None created_at=datetime.datetime(2024, 6, 24, 20, 54, 30, 606381, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2024, 6, 24, 22, 21, 24, 387500, tzinfo=datetime.timezone.utc)
lyft_2021.pdf
document_id=UUID('c5abc0b7-b9e5-54d9-b3d3-fdb14af4d065') version='v0' size_in_bytes=1440303 metadata={'title': 'lyft_2021.pdf', 'source': 'data/lyft_2021.pdf', 'user_id': None} user_id=None created_at=datetime.datetime(2024, 

## Document Chunks

R2R enables retrieval of specific document chunks and associated metadata. To fetch chunks for a particular document:

In [ ]:
document_uuid = 'b722f1ec-b90e-5ed8-b7c8-c768e8b323cb'

document_chunks = r2r_app.document_chunks(document_uuid)

for chunk in document_chunks:
  print(chunk)

{'text': 'An NFT That Saves Lives', 'title': 'pg_essay_4.html', 'source': 'data/pg_essay_4.html', 'version': 'v1', 'chunk_order': 0, 'document_id': 'b722f1ec-b90e-5ed8-b7c8-c768e8b323cb', 'extraction_id': '40a77d4f-d5a9-5d8d-88cd-b6865e37a818'}
{'text': "May 2021Noora Health, a nonprofit I've\nsupported for years, just launched\na new NFT. It has a dramatic name, Save Thousands of Lives,\nbecause that's what the proceeds will do.Noora has been saving lives for 7 years. They run programs in\nhospitals in South Asia to teach new mothers how to take care of\ntheir babies once they get home. They're in 165 hospitals now. And\nbecause they know the numbers before and after they start at a new\nhospital, they can measure the impact they have. It is massive.", 'title': 'pg_essay_4.html', 'source': 'data/pg_essay_4.html', 'version': 'v1', 'chunk_order': 1, 'document_id': 'b722f1ec-b90e-5ed8-b7c8-c768e8b323cb', 'extraction_id': '40a77d4f-d5a9-5d8d-88cd-b6865e37a818'}
{'text': 'For every 1000 li

## Update Documents

R2R provides robust document update capabilities through two main endpoints: update_documents and update_files. These endpoints allow for seamless updating of existing documents while maintaining version control.

Key features of the update process:

- Automatic versioning: When updating a document, R2R automatically increments the version (e.g., from “v0” to “v1”).

- Metadata preservation: The update process maintains existing metadata while allowing for updates.

- Content replacement: The new document content completely replaces the old content in the order shown below

- Ingest the new version of the document
Delete the old version

In [ ]:
ret = r2r_app.update_documents([documents[0]])
print(ret)

2024-06-24 22:21:58,836 - INFO - r2r.main.services.ingestion_service - Deleting documents which match on these keys and values: (['document_id', 'version'], ['c9bdbac7-0ea3-5c9e-b590-018bd09b127b', 'v0'])


Document(s) c9bdbac7-0ea3-5c9e-b590-018bd09b127b updated.


## Delete Documents

R2R supports flexible document deletion based on various metadata fields. To delete a document by its ID:

In [ ]:
delete_keys = ['document_id']
delete_values = [str(documents[0].id)]

ret = r2r_app.delete(delete_keys, delete_values)
print(ret)

2024-06-24 22:22:00,426 - INFO - r2r.main.services.management_service - Deleting entries with metadata: document_id=c9bdbac7-0ea3-5c9e-b590-018bd09b127b


Documents ['c9bdbac7-0ea3-5c9e-b590-018bd09b127b'] deleted successfully.


# AI Powered Search

R2R offers powerful search capabilities, including vector search, hybrid search, and knowledge graph-enhanced search. These features allow for more accurate and contextually relevant information retrieval.

## Basic Vector Search

To perform a basic vector search using RAG, execute the following command:

In [ ]:
# same as the previous example

from r2r import VectorSearchSettings

# search_query = "Who was Aristotle?"
# search_query = "Who was the Kingslayer?"
search_query = "What was uber's profit in 2020?"

search_settings = VectorSearchSettings(
                    search_filters=None,
                    search_limit=3,
                    do_hybrid_search=False,
                )

retrieved_chunks = r2r_app.search(search_query, search_settings)
for chunk in retrieved_chunks['vector_search_results']:
    print(f"Score: {round(chunk['score'], 2)}\nText: {chunk['metadata']['text']}\n{'-'*40}")

Score: 0.74
Text: Net
 loss attributable to Uber Technologies, Inc. was $496 million, a 93% improvement year-over-year, driven by a $1.6 billion pre-tax gain on the sale of ourATG
 Business to Aurora, a $1.6 billion pre-tax  net benefit relating to Ubers equity investments, as  well as reductions in our fixed cost structure and increasedvariable cost effi
ciencies. Net loss attributable to Uber Technologies, Inc. also included $1.2 billion of stock-based compensation expense.Adjusted
----------------------------------------
Score: 0.67
Text: the year ended December 31, 2020, gain on business divestitures, net represented a $154 million gain on the sale of our Uber Eats India operationsto Zomato
 recognized in the first quarter of 2020 and a $77 million gain on the sale of our European Freight Business to sennder GmbH (Sennder) recognized inthe fourth quarter of 2020, pa
rtially offset by a $27 million loss on the sale of our JUMP operations to Lime recognized in the second quarter of 2

## Hybrid Search

R2R supports hybrid search, which combines traditional keyword-based search with vector search for improved results.

In [ ]:
# same as the previous example

from r2r import VectorSearchSettings

# search_query = "Who was Aristotle?"
# search_query = "Who was the Kingslayer?"
search_query = "What was uber's profit in 2020?"

search_settings = VectorSearchSettings(
                    search_filters=None,
                    search_limit=3,
                    do_hybrid_search=True,
                )

retrieved_chunks = r2r_app.search(search_query, search_settings)
for chunk in retrieved_chunks['vector_search_results']:
    print(f"Text: {chunk['metadata']['text']}\n{'-'*40}")

Text: Net
 loss attributable to Uber Technologies, Inc. was $496 million, a 93% improvement year-over-year, driven by a $1.6 billion pre-tax gain on the sale of ourATG
 Business to Aurora, a $1.6 billion pre-tax  net benefit relating to Ubers equity investments, as  well as reductions in our fixed cost structure and increasedvariable cost effi
ciencies. Net loss attributable to Uber Technologies, Inc. also included $1.2 billion of stock-based compensation expense.Adjusted
----------------------------------------
Text: the year ended December 31, 2020, gain on business divestitures, net represented a $154 million gain on the sale of our Uber Eats India operationsto Zomato
 recognized in the first quarter of 2020 and a $77 million gain on the sale of our European Freight Business to sennder GmbH (Sennder) recognized inthe fourth quarter of 2020, pa
rtially offset by a $27 million loss on the sale of our JUMP operations to Lime recognized in the second quarter of 2020.During
-------------

## Knowledge Graph Search

R2R integrates knowledge graph capabilities to enhance search results with structured relationships. To utilize knowledge graph search:



In [ ]:
# check KG search cookbook to create a knowledge graph before running this!

# search_settings = KGSearchSettings(
#                     search_limit=3,
#                     use_kg_search=True,
#                 )

# retrieved_chunks = r2r_app.search(search_query, search_settings)

# Retrieval Augmented Generation (RAG)

R2R is built around a comprehensive Retrieval-Augmented Generation (RAG) engine, allowing you to generate contextually relevant responses based on your ingested documents. The RAG process combines the search functionality with language model generation to produce more accurate and informative answers.

## Basic RAG

To generate a response using RAG, use the following command:

In [ ]:
from r2r import GenerationConfig

rag_query = "Who was Aristotle?"

rag_generation_config = GenerationConfig(
            model="gpt-4o",
            stream=False,
            )

rag_result = r2r_app.rag(
            query=rag_query,
            rag_generation_config=rag_generation_config,
        )

rag_result[0].choices[0].message.content

'Aristotle (Greek: Ἀριστοτέλης Aristotélēs, pronounced [aristotélɛːs]; 384–322 BC) was an Ancient Greek philosopher and polymath. His extensive writings covered a broad range of subjects including the natural sciences, philosophy, linguistics, economics, politics, psychology, and the arts. He founded the Peripatetic school of philosophy in the Lyceum in Athens and initiated the wider Aristotelian tradition, which laid the groundwork for the development of modern science [1].'

## Streaming RAG

R2R also supports streaming RAG responses, which can be useful for real-time applications. To use streaming RAG:




In [ ]:
from r2r import GenerationConfig

rag_query = "Who was Aristotle?"

rag_generation_config = GenerationConfig(
            model="gpt-4o",
            stream=True,
            )

rag_result = r2r_app.rag(
            query=rag_query,
            rag_generation_config=rag_generation_config,
        )

async def _stream_response():
    async for chunk in rag_result:
        print(chunk, end="", flush=True)

asyncio.run(_stream_response())

<search>"{\"id\":\"dac63aab-7c54-52dd-a496-737193ec43a7\",\"score\":0.774121238343654,\"metadata\":{\"text\":\"Aristotle[A] (Greek: \u1f08\u03c1\u03b9\u03c3\u03c4\u03bf\u03c4\u03ad\u03bb\u03b7\u03c2 Aristot\u00e9l\u0113s, pronounced [aristot\u00e9l\u025b\u02d0s]; 384\u2013322 BC) was an Ancient Greek philosopher and polymath. His writings cover a broad range of subjects spanning the natural sciences, philosophy, linguistics, economics, politics, psychology, and the arts. As the founder of the Peripatetic school of philosophy in the Lyceum in Athens, he began the wider Aristotelian tradition that followed, which set the groundwork for the development of modern science.\",\"title\":\"aristotle_v2.txt\",\"source\":\"data/aristotle_v2.txt\",\"version\":\"v1\",\"chunk_order\":0,\"document_id\":\"dbf69e0a-3d41-5140-b20d-017cd6c01ca7\",\"extraction_id\":\"f11dbdc7-9e5c-5207-abf4-e69ed5caabbd\",\"associatedQuery\":\"Who was Aristotle?\"}}","{\"id\":\"ff76490a-e11f-5f8b-94ad-23ee06c2e7ef\",\"sc

## Customizing RAG

R2R offers extensive customization options for its Retrieval-Augmented Generation (RAG) functionality:

Search Settings: Customize vector and knowledge graph search parameters using VectorSearchSettings and KGSearchSettings.

Generation Config: Fine-tune the language model’s behavior with GenerationConfig, including:

Temperature, top_p, top_k for controlling randomness
Max tokens, model selection, and streaming options
Advanced settings like beam search and sampling strategies
Multiple LLM Support: Easily switch between different language models and providers:

OpenAI models (default)
Anthropic’s Claude models
Local models via Ollama
Any provider supported by LiteLLM
Example of customizing the model:




# Observability and Analytics

R2R provides robust observability and analytics features, allowing you to monitor system performance, track usage patterns, and gain insights into your RAG application’s behavior.

## Logging

R2R automatically logs various events and metrics during its operation. You can access these logs using the logs command:

In [ ]:
log_type_filter = "search" # rag, ingestion
logs = r2r_app.logs(log_type_filter)
for log in logs:
    print(log)

{'run_id': UUID('ca83a0db-4ebd-4f51-9bba-866123137098'), 'run_type': 'search', 'entries': [{'key': 'search_latency', 'value': '0.91'}, {'key': 'search_results', 'value': '["{\\"id\\":\\"687e7151-33a3-59e0-b377-094d99fc4cd0\\",\\"score\\":1.0,\\"metadata\\":{\\"text\\":\\"Net\\\\n loss attributable to Uber Technologies, Inc. was $496 million, a 93% improvement year-over-year, driven by a $1.6 billion pre-tax gain on the sale of ourATG\\\\n Business to Aurora, a $1.6 billion pre-tax  net benefit relating to Ubers equity investments, as  well as reductions in our fixed cost structure and increasedvariable cost effi\\\\nciencies. Net loss attributable to Uber Technologies, Inc. also included $1.2 billion of stock-based compensation expense.Adjusted\\",\\"title\\":\\"uber_2021.pdf\\",\\"source\\":\\"data/uber_2021.pdf\\",\\"version\\":\\"v1\\",\\"chunk_order\\":739,\\"document_id\\":\\"c996e617-88a4-5c65-ab1e-948344b18d27\\",\\"extraction_id\\":\\"9592e666-dd2c-5419-ab98-ddb190a8a972\\",\\"

## Analytics



In [ ]:
from r2r import FilterCriteria, AnalysisTypes

filter_criteria = FilterCriteria(filters={"search_latencies": "search_latency"})
analysis_types = AnalysisTypes(analysis_types={"search_latencies": ["basic_statistics", "search_latency"]})

analytics = r2r_app.analytics(filter_criteria, analysis_types)

analytics

{'filtered_logs': {'search_latencies': [{'timestamp': '2024-06-24 22:22:09',
    'log_id': UUID('ca83a0db-4ebd-4f51-9bba-866123137098'),
    'key': 'search_latency',
    'value': '0.91',
    'rn': 3},
   {'timestamp': '2024-06-24 22:22:03',
    'log_id': UUID('06dff51e-3871-4cd4-a9d6-daf83bafa788'),
    'key': 'search_latency',
    'value': '0.42',
    'rn': 3},
   {'timestamp': '2024-06-24 22:21:33',
    'log_id': UUID('cc83fb4b-219f-408a-bb88-cae829e3dfb0'),
    'key': 'search_latency',
    'value': '0.47',
    'rn': 3},
   {'timestamp': '2024-06-24 21:46:56',
    'log_id': UUID('a27c26b3-1746-4db3-acde-7bb3a36a6ea5'),
    'key': 'search_latency',
    'value': '0.94',
    'rn': 3},
   {'timestamp': '2024-06-24 21:45:36',
    'log_id': UUID('d106495c-6798-4cf9-a60c-dc726770859e'),
    'key': 'search_latency',
    'value': '0.99',
    'rn': 3},
   {'timestamp': '2024-06-24 21:45:30',
    'log_id': UUID('88413bca-8d18-4e48-8e38-39306c3ee4e6'),
    'key': 'search_latency',
    'value': '

## Custom Analytics

R2R’s analytics system is flexible and allows for custom analysis. You can specify different filters and analysis types to focus on specific aspects of your application’s performance. For example:

Analyze RAG latencies
Track usage patterns by user or document type
Monitor error rates and types
Assess the effectiveness of different LLM models or configurations
To perform custom analytics, modify the filters and analysis_types parameters in the analytics command to suit your specific needs.

# User Management

R2R provides powerful user management capabilities, allowing you to track and manage documents on a per-user basis. Refer to the documentation for more details!

You can:

- Retrieve an overview for your users and get analytics
- Perform user specific RAG queries